In [1]:
import pandas as pd
import numpy as np

In [2]:
people_df = pd.read_csv(r"C:\Users\19692\Downloads\CSE560-Project\sampled_data\people_sampled.tsv", sep="\t")
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635919 entries, 0 to 635918
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   nconst             635919 non-null  object
 1   primaryName        635919 non-null  object
 2   birthYear          635919 non-null  object
 3   deathYear          635919 non-null  object
 4   primaryProfession  635919 non-null  object
 5   knownForTitles     635919 non-null  object
dtypes: object(6)
memory usage: 29.1+ MB


In [3]:
# replace \N to ''
people_df['birthYear'] = people_df['birthYear'].replace(r'\N','')
people_df['deathYear'] = people_df['deathYear'].replace(r'\N','')
people_df['primaryProfession'] = people_df['primaryProfession'].replace(r'\N','')
people_df['knownForTitles'] = people_df['knownForTitles'].replace(r'\N','')


In [4]:
# drop na rows for not null columns
people_df[["nconst", "primaryName","primaryProfession"]].dropna()
people_df = people_df[people_df["primaryName"].isna()==False]
people_df = people_df[people_df["primaryProfession"].isna()==False]

In [5]:
# replace , and ; to /
people_df['primaryName'] = people_df['primaryName'].str.replace(r',','/', regex=False)
people_df['primaryName'] = people_df['primaryName'].str.replace(r';','/', regex=False)

In [85]:
import psycopg2
import io

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres.rnxnnqsyjcircloxrryq",
    password="cse560courseproject",
    host="aws-0-us-east-1.pooler.supabase.com",
    port=6543
)
cur = conn.cursor()

In [86]:
conn.rollback()

## Get People relation

In [44]:
people = people_df[['nconst','primaryName','birthYear','deathYear']]

In [48]:

# for tv_movie relation
buffer = io.StringIO()
people.to_csv(buffer, index=False, header=False)
buffer.seek(0)

0

In [49]:

# upload data to tv_movie
cur.copy_from(buffer, 'people', sep=',', null='')
conn.commit()

## Get Profession

In [50]:
profession_set = set()
for i, row in people_df.iterrows():
    profession = row['primaryProfession']
    profession_arr = profession.split(",")
    for p in profession_arr:
        profession_set.add(p)

In [60]:
profession_df = pd.DataFrame(list(profession_set), columns=['profession_name'])

In [61]:
profession_df = profession_df[
    profession_df['profession_name'].notna() & 
    (profession_df['profession_name'].str.strip() != '')
]

In [69]:
# for tv_movie relation
buffer = io.StringIO()
profession_df.to_csv(buffer, index=False, header=False)
buffer.seek(0)

0

In [71]:
conn.rollback()

In [72]:
cur.copy_from(buffer, 'profession', sep=',', null='', columns=('profession_name',))
conn.commit()


## Get profession_people

In [87]:
people_df['primaryProfession'] = people_df['primaryProfession'].str.split(",")
people_df_exploded = people_df.explode('primaryProfession')
people_df_exploded

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,actor,"tt0072308,tt0050419,tt0027125,tt0031983"
0,nm0000001,Fred Astaire,1899,1987,miscellaneous,"tt0072308,tt0050419,tt0027125,tt0031983"
0,nm0000001,Fred Astaire,1899,1987,producer,"tt0072308,tt0050419,tt0027125,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,actress,"tt0037382,tt0075213,tt0117057,tt0038355"
1,nm0000002,Lauren Bacall,1924,2014,soundtrack,"tt0037382,tt0075213,tt0117057,tt0038355"
...,...,...,...,...,...,...
635917,nm9993693,Apsara Rani,1996,,actress,"tt12856788,tt13847502,tt8302382,tt25385692"
635917,nm9993693,Apsara Rani,1996,,archive_footage,"tt12856788,tt13847502,tt8302382,tt25385692"
635918,nm9993694,Chinmay Mishra,,,actor,"tt18361688,tt18687502,tt8737752,tt9083282"
635918,nm9993694,Chinmay Mishra,,,director,"tt18361688,tt18687502,tt8737752,tt9083282"


In [100]:
profession_df = profession_df.reset_index().rename(columns={'index':'profession_id'})

In [101]:
people_df_exploded = people_df_exploded.rename(columns={'primaryProfession':'profession_name'})

In [102]:
people_profession = people_df_exploded.merge(profession_df,on='profession_name', how='inner')


In [103]:
people_profession

,nconst,primaryName,birthYear,deathYear,profession_name,knownForTitles,profession_id
0,nm0000001,Fred Astaire,1899,1987,actor,"tt0072308,tt0050419,tt0027125,tt0031983",11
1,nm0000001,Fred Astaire,1899,1987,miscellaneous,"tt0072308,tt0050419,tt0027125,tt0031983",19
2,nm0000001,Fred Astaire,1899,1987,producer,"tt0072308,tt0050419,tt0027125,tt0031983",1
3,nm0000002,Lauren Bacall,1924,2014,actress,"tt0037382,tt0075213,tt0117057,tt0038355",30
4,nm0000002,Lauren Bacall,1924,2014,soundtrack,"tt0037382,tt0075213,tt0117057,tt0038355",5
...,...,...,...,...,...,...,...
1169210,nm9993693,Apsara Rani,1996,,actress,"tt12856788,tt13847502,tt8302382,tt25385692",30
1169211,nm9993693,Apsara Rani,1996,,archive_footage,"tt12856788,tt13847502,tt8302382,tt25385692",18
1169212,nm9993694,Chinmay Mishra,,,actor,"tt18361688,tt18687502,tt8737752,tt9083282",11
1169213,nm9993694,Chinmay Mishra,,,director,"tt18361688,tt18687502,tt8737752,tt9083282",24


In [104]:
people_profession = people_profession[['profession_id', 'nconst']]

In [106]:
# for tv_movie relation
buffer = io.StringIO()
people_profession.to_csv(buffer, index=False, header=False)
buffer.seek(0)

0

In [107]:
cur.copy_from(buffer, 'profession_people', sep=',', null='')
conn.commit()


# Get famouse_work Relation

In [115]:
people_df['knownForTitles'] = people_df['knownForTitles'].str.split(',')

In [118]:
people_df = people_df.explode('knownForTitles')

In [121]:
famous_work = people_df[['nconst','knownForTitles']]

In [ ]:
# for actors without famous work, replace empty column with 'N/A'
famous_work = people_df[['nconst', 'knownForTitles']].copy()
famous_work['knownForTitles'] = famous_work['knownForTitles'].fillna('N/A')
famous_work['knownForTitles'] = famous_work['knownForTitles'].replace(r'^\s*$', 'N/A', regex=True)

In [ ]:
# make it only contain record where knowForTitles is also in TV_Moive Relation
title_basics_df = pd.read_csv(r"C:\Users\19692\Downloads\CSE560-Project\sampled_data\TV_Movie_sampled.tsv", sep="\t")
tconst_set = set(title_basics_df['tconst'])
famous_work = famous_work[famous_work['knownForTitles'].isin(tconst_set)]

In [ ]:
# change order of the two columns
famous_work = famous_work[['knownForTitles', 'nconst']]

In [149]:
buffer = io.StringIO()
famous_work.to_csv(buffer, index=False, header=False)
buffer.seek(0)

0

In [150]:
conn.rollback()

In [151]:
cur.copy_from(buffer, 'famous_work', sep=',', null='')
conn.commit()